# Генераторы
Марк Лутц "Изучаем Python", том 1, глава 20.



---



В Python списковые включения и генераторы - замечательные механизмы, способные серьезно упрощать программный код.

Генераторы позволяют приостановить или продолжить работы. Генератор возвращает итератор, по которому можно проходить пошагово, получая доступ к одному значению с каждой итерацией.

Есть две языковые конструкции реализующие генераторы:

*   **Генераторные функции** (доступные, начиная с версии Python 2.3) записываются как нормальные операторы *def*, но в них применяются операторы *yield*, чтобы возвращать по одному результату за раз, приостанавливать выполнение с сохранением состояния и возобновлять его между выдачами.
*   **Генераторные выражения** (доступные, начиная с версии Python 2.4) похожи на списковые включения, описанные в предыдущем докладе, но они не строят результирующий список, а возвращают объекты, которые производят результаты
по запросу.

Из-за того, что ни та, ни другая конструкция не создает сразу весь результирующий список, они экономят пространство памяти и позволяют распределить время вычислений по запросам результатов.







## Генераторные функции

Генераторные функции во многих отношениях похожи на обычные функции и
фактически записываются с помощью обычных операторов. Однако при их создании они специально компилируются в объект, который поддерживает протокол итерации. Когда генераторные функции вызываются, они возвращают не результат, а генератор результатов, который может появляться в любом итерационном контексте.
Основным отличием генераторных функций от обычных является оператор *yield*.

In [ ]:
# Генераторная функция
def gensquares(N):
  for i in range(N):
    yield i**2 # Позже возобновить здесь выполнение

for i in gensquares(5): # Возобновление выполнения функции
  print(i, end=' : ') # Вывод последнего выданного значения

0 : 1 : 4 : 9 : 16 : 

In [ ]:
x = gensquares(5)
x

<generator object gensquares at 0x7fda57e18750>

In [ ]:
next(x)

TypeError: ignored

In [ ]:
x.__next__()

StopIteration: ignored

### Для чего используются генераторные функции?

Реализация логики из предыдущей генераторной функции на других конструкциях:

In [ ]:
# Обычная функция
def buildsquares(n):
  res = []
  for i in range(n):
    res.append(i ** 2)
  return res

for x in buildsquares(5):
  print(x, end=" : ")

0 : 1 : 4 : 9 : 16 : 

In [ ]:
# Функция map (аналогичный код)
for x in map((lambda n: n**2), range(5)):
  print(x, end=" : ")

0 : 1 : 4 : 9 : 16 : 

В крупных программах генераторы могут быть лучше в плане памяти
и производительности. Они позволяют функциям избежать выполнения всей работы
заранее, что особенно полезно, когда результирующие списки большие или получение
каждого значения требует длительных вычислений. Генераторы распределяют время,
необходимое для производства серии значений, по всем итерациям цикла.

Кроме того, для усложненных сценариев генераторы способны предложить более простую альтернативу ручному сохранению состояния между итерациями в объектах классов — генераторы обеспечивают автоматическое сохранение и
восстановление переменных, доступных в области видимости функций.

### Метод *send()*

В версии Python 2.5 к протоколу генераторных функций был добавлен метод *send*.
Он осуществляет переход на следующий элемент в серии результатов, в точности как
__next __, но также снабжает вызывающий код возможностью взаимодействия с генератором для влияния на его работу.

In [ ]:
def gen():
  for i in range(10):
    print('i1 = ' + str(i))
    x = yield i
    print('x = ' + str(x))
    print('i2 = ' + str(i))
G = gen()

Формально *yield* теперь имеет форму не оператора, а выражения, которое возвращает элемент, переданный *send*.

Метод *send* можно применять, например, для написания генератора, который
разрешает прекращать свою работу за счет отправки кода завершения, либо изменять
направление путем передачи новой позиции в данных, обрабатываемых внутри генератора.

In [ ]:
G.send(77)

TypeError: ignored

In [ ]:
next(G) # Сначала должен вызываться next() , чтобы запустить генератор

x = None
i2 = 0
i1 = 1


1

In [ ]:
G.send(77)

x = 77
i2 = 1
i1 = 2


2

In [ ]:
G.send(88)

x = 88
i2 = 2
i1 = 3


3

In [ ]:
next(G)

x = None
i2 = 6
i1 = 7


7

### Расширение *from*
Также следует отметить, что в Python 3.3 было введено расширение *yield* — конструкция *from*, которая позволяет генераторам делегировать работу вложенным генераторам.

In [ ]:
# с использованием цикла for
def iter_for():
  for x in range(3):
    yield x

for i in iter_for():
  print(i)

0
1
2


In [ ]:
# с использованием from
def iter_from():
  yield from [0, 1, 2]

for i in iter_from():
  print(i)

0
1
2


In [ ]:
def iter_from():
  yield from [0, 1, 2]
iters = iter_from()

In [ ]:
next(iters)

StopIteration: ignored

In [ ]:
# с использованием from и спискового включения
def iter_from():
  yield from [x for x in range(3)]

for i in iter_from():
  print(i)

0
1
2


### Дополнительные методы *close()* и *throw()*
*.close()* — останавливает выполнение генератора;

*.throw()* — генератор бросает исключение.

In [ ]:
def f_gen():
  n = 1
  while True:
    yield n**2
    n += 1
  
generator1 = f_gen()
generator2 = f_gen()

for i in generator1:
  print(i)
  if i > 10:
    generator1.close()

print('\n')

for i in generator2:
  print(i)
  if i > 10:
    generator2.throw(Exception("Конец!"))

1
4
9
16


1
4
9
16


Exception: ignored

## Генераторные выражения

Из-за того, что отложенные вычисления генераторных функций оказались настолько удобными, со временем они распространились на другие инструменты. В Python 2.Х и З.Х понятия итерируемых объектов и списковых включений были объединены в новый инструмент — генераторные выражения. Синтаксически генераторные выражения похожи на нормальные списковые включения и поддерживают весь их синтаксис, в том числе фильтры if и вложение циклов, но они помещаются в круглые скобки, а не в квадратные.

In [ ]:
print([х**2 for х in range(4)]) # Списковое включение: строит список [О, 1, 4, 9]

print((х**2 for х in range(4))) # Генераторное выражение:создает итерируемый объект

[0, 1, 4, 9]
<generator object <genexpr> at 0x7fda4f5631d0>


In [ ]:
G = (х**2 for х in range(4))

In [ ]:
next(G)

StopIteration: ignored

In [ ]:
''.join(х.upper() for х in 'ааа,bbb,ссс'.split(','))

'АААBBBССС'

### Для чего используются генераторные выражения?

В точности как генераторные функции генераторные выражения обеспечивают
оптимизацию **расхода памяти** — они не требуют создания сразу всего результирующего списка, что происходит в случае спискового включения в квадратных скобках. 

Также подобно генераторным функциям они разделяют работу по выпуску результатов на небольшие временные интервалы — результаты выдаются постепенно вместо того, чтобы заставлять вызывающий код ожидать создания полного набора в единственном вызове. 

С другой стороны, на практике генераторные выражения могут выполняться несколько медленнее списковых включений, а потому их лучше всего применять для очень крупных результирующих наборов или в приложениях, которые не могут ожидать генерации полных результатов.

1. Проще синтаксис

In [ ]:
print(list(map(abs, (-1, -2, 3, 4)))) # map применяет функцию abs для кортежа

print(list(abs(x) for x in (-1, -2, 3, 4))) # ген.выражение применяет функцию abs для кортежа

print(list(map(lambda x: x*2, (1, 2, 3, 4)))) # map использует НЕ функцию

print(list(x*2 for x in (1, 2, 3, 4))) # ген.выражение использует НЕ функцию

[1, 2, 3, 4]
[1, 2, 3, 4]
[2, 4, 6, 8]
[2, 4, 6, 8]


2. Экономия памяти

In [ ]:
line = 'aaa,bbb,ссс'
print(''.join([х.upper() for х in line.split (',')])) # создание бессмысленного списка 

print(''.join(х.upper() for х in line.split(','))) # сразу генерируется строка

AAABBBССС
AAABBBССС


### Вложенные генераторы

Вложенные **списковые включения** создают дополнительные временные списки, расходуя память.

Вложенные **функции map** сложнее в представлении кода.

Вложенные **генераторные выражения** не формируют временные списки и проще в представлении кода, чем map.




In [ ]:
print([х*2 for х in [abs(x) for x in (-1, -2, 3, 4)]]) # Вложенные включения

print(list(map(lambda x: x*2, map(abs, (-1, -2, 3, 4))))) # Вложенные map

print(list(x*2 for x in (abs(x) for x in (-1, -2, 3, 4)))) # Вложенные генераторы

[2, 4, 6, 8]
[2, 4, 6, 8]
[2, 4, 6, 8]


### Оператор *if*

Генераторные выражения также поддерживают весь обычный синтаксис списковых включений, в том числе конструкции *if*, которые работают подобно вызову *filter*. 

In [ ]:
line = 'аа bbb а'
print(''.join(x for x in line.split() if len(x) > 1)) # Генератор c конструкцией if

print(''.join(filter(lambda x: len(x) > 1, line.split()))) # Подобный вызов filter

ааbbb
ааbbb


# Генераторные функции или генераторные выражения


### Генераторные функции
Определение *def*, содержащее оператор *yield*, превращается в генераторную функцию. При вызове она возвращает новый объект генератора с автоматическим сохранением локальной области видимости и местоположения в коде, автоматически созданным методом *__ iter__*, который просто возвращает сам объект, и автоматически созданным методом *__ next__* (*next* в Python 2.Х), который запускает функцию или возобновляет ее выполнение с места, где она находилась в последний раз, и инициирует исключение *StopIteration*, когда выпуск результатов завершен.

### Генераторные выражения
Выражение включения, помещенное в круглые скобки, известно как генераторное выражение. Оно возвращает новый объект генератора с таким же автоматически созданным интерфейсом в виде методов и сохранением состояния, как у генераторной функции, т.е. с методом *__ iter__*, просто возвращающим сам объект, и методом *__ next__* (*next* в Python 2.Х), который запускает неявный цикл или возобновляет его выполнение с места, где он находился в последний раз, и инициирует исключение *StopIteration*, когда выпуск результатов завершен.

Совокупным эффектом является выпуск результатов по запросу в итерационных контекстах, которые задействуют такие интерфейсы 
автоматически.

Эквивалетный код:

In [ ]:
line = 'аа bbb с'

print(''.join(x.upper() for x in line.split() if len(x) > 1)) # Выражение

def gensub(line): # Функция
  for x in line.split():
    if len(x) > 1:
      yield x.upper()
      
print(''.join(gensub(line)))

ААBBB
ААBBB


## Генераторы являются объектами с одиночной итерацией

Тонкий, но важный момент: генераторные функции и генераторные выражения представляют собой итераторы и потому поддерживают только одну активную итерацию — в отличие от ряда встроенных типов нельзя иметь множество итераторов, каждый из которых находится в отличающейся позиции внутри набора результатов.

В случае выполнения итерации по потоку результатов с помощью множества итераторов все они будут указывать на ту же самую позицию:

In [ ]:
G = (с*4 for с in 'SPAM') # Создать новый генератор
print(next(G))

I1 = G.__iter__() # Первый итератор
print(next(I1))

I2 = G.__iter__() # Второй итератор находится в той же самой позиции!
print(next(I2))

SSSS
PPPP
AAAA


Более того, как только любая итерация доходит до завершения, все результаты
оказываются израсходованными — чтобы начать сначала, нам придется создать новый
генератор:

In [ ]:
print(next(I1)) # Собирает оставшиеся элементы I1
print(next(I2)) # Другие итераторы тоже израсходуются

MMMM


StopIteration: ignored

In [ ]:
I3 = G.__iter__() # То же самое касается новых итераторов
print(next(I3))

StopIteration: ignored

In [ ]:
I3 = (с*4 for с in 'SPAM') # Новый генератор, чтобы начать заново
print(next(I3))

SSSS


# Не злоупотребляйте генераторами

Генераторы — довольно сложный инструмент, который возможно лучше трактовать как необязательную тему, если не учитывать тот факт, что они буквально пронизывают язык Python, особенно в линейке Python З.Х.

В целом те же самые предостережения, которые давались в отношении списковых включений, применимы и здесь: 

> Не усложняйте свой код определяемыми пользователем генераторами, когда это неоправданно

Веские причины использовать такие инструменты могут отсутствовать, особенно для небольших программ и наборов данных.
В подобных ситуациях достаточно будет простых списков результатов, которые легче для понимания, автоматически подвергаются сборке мусора и могут выпускаться
быстрее. С такими расширенными инструментами, как генераторы, полагающимся на неявную “магию”, может быть интересно экспериментировать, но им не место в реальном коде, который должен применяться другими, кроме случаев, когда это явно оправданно.

Или прибегнем к девизам, выводимым *import this*: 

> Явное лучше неявного





In [ ]:
import this

Чем более явно код сообщает о своем поведении, тем более вероятно, что следующий программист сумеет его понять. Сказанное напрямую касается генераторов, чье неявное поведение вполне может оказаться трудным для понимания, нежели более ясные альтернативы. 

> Всегда сохраняйте код простым, если только он не обязан быть сложным!